In [1]:
!nvidia-smi

Thu Mar 28 12:51:50 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:D8:00.0 Off |                  Off |
| N/A   27C    P0    24W / 150W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#import transformer_lens

import sys
from fancy_einsum import einsum

import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformers import AutoTokenizer, pipeline, logging, AutoModelForCausalLM, AutoConfig
from transformer_lens import ActivationCache, HookedTransformer
from transformer_lens.loading_from_pretrained import *

from huggingface_hub import notebook_login
from datasets import Dataset
import pandas as pd
import transformers
import torch
import einops  # Make sure einops is imported

from neel_plotly import line, imshow, scatter
import transformer_lens.patching as patching
import circuitsvis as cv
from pathlib import Path
import json

In [3]:
from load_HookedTransformer import load_HookedTransformer, customized_get_pretrained_model_config


In [5]:
torch.set_grad_enabled(False)

#model,tokenizer = load_HookedTransformer('gpt2-small')
model,tokenizer = load_HookedTransformer('llama-7b')
#model,tokenizer = load_HookedTransformer('gpt2-small')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model llama-7b into HookedTransformer


In [ ]:


torch.set_grad_enabled(False)


model_type="gpt2-large"

model_paths = {
        'gpt2-small': "../Models/gpt2-small",
        'gpt2-medium': "../Models/gpt2-medium",
        'gpt2-large': "../Models/gpt2-large",
        'gpt2-xl': "../Models/gpt2-xl",
    }
model_cfg={
        'gpt2-small':{
        "d_model": 768,
        "n_layers": 12,
        "d_mlp": 3072,
        "d_head": 64,
        "n_heads": 12,
        "n_ctx": 1024,
        "d_vocab": 50257,
        "tokenizer_name": "gpt2",
        "act_fn": "gelu_new",
        "attn_only": False,
        "final_rms": False,
        "original_architecture":"GPT2LMHeadModel",
        },
        'gpt2-medium':{
        "d_model": 1024,
        "n_layers": 24,
        "d_mlp": 4096,
        "d_head": 64,
        "n_heads": 16,
        "n_ctx": 1024,
        "d_vocab": 50257,
        "tokenizer_name": "gpt2",
        "act_fn": "gelu_new",
        "attn_only": False,
        "final_rms": False,
        "original_architecture":"GPT2LMHeadModel",
        },
            'gpt2-large':{
        "d_model": 1280,
        "n_layers": 36,
        "d_mlp": 5120,
        "d_head": 64,
        "n_heads": 20,
        "n_ctx": 1024,
        "d_vocab": 50257,
        "tokenizer_name": "gpt2",
        "act_fn": "gelu_new",
        "attn_only": False,
        "final_rms": False,
        "original_architecture": "GPT2LMHeadModel",
    },
            'gpt2-xl':{
        "d_model": 1600,
        "n_layers": 48,
        "d_mlp": 6400,
        "d_head": 64,
        "n_heads": 25,
        "n_ctx": 1024,
        "d_vocab": 50257,
        "tokenizer_name": "gpt2",
        "act_fn": "gelu_new",
        "attn_only": False,
        "final_rms": False,
        "original_architecture": "GPT2LMHeadModel",
    },     
}
if True:
    tokenizer_path = model_paths[model_type]
    model_path = Path(model_paths.get(model_type))
    if not model_path:
        raise ValueError(f"Unsupported model type: {model_type}")

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    
    #hf_config = AutoConfig.from_pretrained(model_path)
    #config = AutoConfig.from_pretrained(model_path+'/')
    #config['model_type']=model_type
    hf_model = AutoModelForCausalLM.from_pretrained(model_path)#, low_cpu_mem_usage=True)
    #hf_model =GPT2Model.from_pretrained(model_path)#, low_cpu_mem_usage=True)
    for param in hf_model.parameters():
        param.requires_grad = False



    
    cfg = model_cfg[model_type]
    cfg= HookedTransformerConfig.from_dict(cfg)

    with open(model_path / 'config.json', 'r') as config_file:
        cfg_loaded=json.load(config_file)
        
    model = HookedTransformer(cfg=cfg, tokenizer=tokenizer, move_to_device=False, default_padding_side="right")

    #state_dict = hf_model.state_dict()
    state_dict = convert_gpt2_weights(hf_model, cfg)
    model.load_and_process_state_dict(
                state_dict,
                fold_ln=True,
                center_writing_weights=True,
                center_unembed=True,
                fold_value_biases=True,
                refactor_factored_attn_matrices=False,
    )
        
    model.move_model_modules_to_device()
    model.set_use_attn_result(True)


In [ ]:

    model_cfg={
        'gpt2-small':{
        "d_model": 768,
        "n_layers": 12,
        "d_mlp": 3072,
        "d_head": 64,
        "n_heads": 12,
        "n_ctx": 1024,
        "d_vocab": 50257,
        "tokenizer_name": "gpt2",
        "act_fn": "gelu_new",
        "attn_only": False,
        "final_rms": False,
        "original_architecture":"GPT2LMHeadModel",
        },
        'gpt2-medium':{
        "d_model": 1024,
        "n_layers": 24,
        "d_mlp": 4096,
        "d_head": 64,
        "n_heads": 16,
        "n_ctx": 1024,
        "d_vocab": 50257,
        "tokenizer_name": "gpt2",
        "act_fn": "gelu_new",
        "attn_only": False,
        "final_rms": False,
        "original_architecture":"GPT2LMHeadModel",
        },
            'gpt2-large':{
        "d_model": 1280,
        "n_layers": 36,
        "d_mlp": 5120,
        "d_head": 64,
        "n_heads": 20,
        "n_ctx": 1024,
        "d_vocab": 50257,
        "tokenizer_name": "gpt2",
        "act_fn": "gelu_new",
        "attn_only": False,
        "final_rms": False,
        "original_architecture": "GPT2LMHeadModel",
    },
            'gpt2-xl':{
        "d_model": 1600,
        "n_layers": 48,
        "d_mlp": 6400,
        "d_head": 64,
        "n_heads": 25,
        "n_ctx": 1024,
        "d_vocab": 50257,
        "tokenizer_name": "gpt2",
        "act_fn": "gelu_new",
        "attn_only": False,
        "final_rms": False,
        "original_architecture": "GPT2LMHeadModel",
    },     
    }
    
    cfg = model_cfg[model_type]
    cfg= HookedTransformerConfig.from_dict(cfg)

    with open(model_path / 'config.json', 'r') as config_file:
        cfg_loaded=json.load(config_file)
        
    model = HookedTransformer(cfg=cfg, tokenizer=tokenizer, move_to_device=False, default_padding_side="right")

    #state_dict = hf_model.state_dict()
    state_dict = convert_gpt2_weights(hf_model, cfg)
    model.load_and_process_state_dict(
                state_dict,
                fold_ln=True,
                center_writing_weights=True,
                center_unembed=True,
                fold_value_biases=True,
                refactor_factored_attn_matrices=False,
    )
        
    model.move_model_modules_to_device()
    model.set_use_attn_result(True)

In [ ]:
model.

In [ ]:
prompts = ['When John and Mary went to the shops, John gave the bag to', 'When John and Mary went to the shops, Mary gave the bag to', 'When Tom and James went to the park, James gave the ball to', 'When Tom and James went to the park, Tom gave the ball to', 'When Dan and Sid went to the shops, Sid gave an apple to', 'When Dan and Sid went to the shops, Dan gave an apple to', 'After Martin and Amy went to the park, Amy gave a drink to', 'After Martin and Amy went to the park, Martin gave a drink to']
answers = [(' Mary', ' John'), (' John', ' Mary'), (' Tom', ' James'), (' James', ' Tom'), (' Dan', ' Sid'), (' Sid', ' Dan'), (' Martin', ' Amy'), (' Amy', ' Martin')]

clean_tokens = model.to_tokens(prompts)
# Swap each adjacent pair, with a hacky list comprehension
corrupted_tokens = clean_tokens[
    [(i+1 if i%2==0 else i-1) for i in range(len(clean_tokens)) ]
    ]
print("Clean string 0", model.to_string(clean_tokens[0]))
print("Corrupted string 0", model.to_string(corrupted_tokens[0]))

answer_token_indices = torch.tensor([[model.to_single_token(answers[i][j]) for j in range(2)] for i in range(len(answers))], device=model.cfg.device)
print("Answer token indices", answer_token_indices)

Clean string 0 <s> When John and Mary went to the shops, John gave the bag to
Corrupted string 0 <s> When John and Mary went to the shops, Mary gave the bag to


AssertionError: Input string:  Mary is not a single token!

In [7]:
def get_logit_diff(logits, answer_token_indices=answer_token_indices):
    if len(logits.shape)==3:
        # Get final logits only
        logits = logits[:, -1, :]
    correct_logits = logits.gather(1, answer_token_indices[:, 0].unsqueeze(1))
    incorrect_logits = logits.gather(1, answer_token_indices[:, 1].unsqueeze(1))
    return (correct_logits - incorrect_logits).mean()

clean_logits, clean_cache = model.run_with_cache(clean_tokens)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)

clean_logit_diff = get_logit_diff(clean_logits, answer_token_indices).item()
print(f"Clean logit diff: {clean_logit_diff:.4f}")

corrupted_logit_diff = get_logit_diff(corrupted_logits, answer_token_indices).item()
print(f"Corrupted logit diff: {corrupted_logit_diff:.4f}")

NameError: name 'answer_token_indices' is not defined

In [ ]:
clean_logits[4,6,10025:10030] , clean_logits[4,13,10025:10030]

In [ ]:
model.embed.state_dict().values()

In [ ]:
model.ln_final.state_dict().values()

In [ ]:
for i,j in enumerate(model.blocks[0].mlp.state_dict().values()):
  print(i,j)